In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Hyperparameters
batch_size = 64
hidden_size = 128
num_classes = 10
learning_rate = 0.001
num_epochs = 5

# Data loader
transform = transforms.ToTensor()
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 14.3MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 342kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 3.18MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 6.90MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
# Model
class BiLSTM2D(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(BiLSTM2D, self).__init__()
        self.hidden_size = hidden_size

        # LSTM for rows (28 sequences of 28 values)
        self.lstm_row = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)

        # LSTM for columns (transpose input)
        self.lstm_col = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)

        # Final classifier
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        # x: [batch, 1, 28, 28]
        x = x.squeeze(1)  # [batch, 28, 28]

        # Process rows
        out_row, (hn_row, _) = self.lstm_row(x)  # [batch, 28, hidden]
        hn_row = hn_row[-1]  # [batch, hidden]

        # Process columns
        x_col = x.transpose(1, 2)  # [batch, 28, 28] -> [batch, 28, 28] still (transpose for columns)
        out_col, (hn_col, _) = self.lstm_col(x_col)
        hn_col = hn_col[-1]  # [batch, hidden]

        # Concatenate features
        features = torch.cat((hn_row, hn_col), dim=1)  # [batch, hidden*2]

        # Final output
        out = self.fc(features)  # [batch, num_classes]
        return out

In [ ]:
# Initialize model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM2D(input_size=28, hidden_size=hidden_size, num_classes=num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')